In [13]:
#Linear Regression
#https://www.tensorflow.org/tutorials/keras/regression

from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [14]:
dataset_path="merged.csv"
column_names = ['idx','Menu Item Number','Menu Item','Sales Count','Sales Total', 
                'Date', 'Day', 'Name of Restaurant']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?",dtype={'idx': object},
                      sep=",", skiprows=[0])
raw_dataset.drop(['idx'], axis=1)
dataset = raw_dataset.copy()
#find unknown values
dataset.isna().sum()
#drop rows with unknown values
dataset = dataset.dropna()
#dataset.tail(30)
dataset.head()

,idx,Menu Item Number,Menu Item,Sales Count,Sales Total,Date,Day,Name of Restaurant
0,7,2429,TO GO,1,0,2019-03-18,0,Origami
1,8,3624,SALMON POKE BOWL,4,29,2019-03-18,0,Origami
2,9,3630,SPCY CRAB MAKI,17,114.75,2019-03-18,0,Origami
3,10,3631,SALM CR CHZ MAKI,8,55.6,2019-03-18,0,Origami
4,11,3632,TUNA MAKI,2,13.5,2019-03-18,0,Origami


In [15]:
#useless idx col
dataset.pop('idx')
#manually clean errors in dataset :(
#day column is string, convert it to int 
dataset['Sales Count'] = dataset['Sales Count'].astype(int)
dataset['Sales Total'] = dataset['Sales Total'].astype(float) #encountered '$-' in csv 115610, manually changed to 0
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset['Day'] = dataset['Day'].astype(int)

#Day column is categorical not numeric
day = dataset.pop('Day')
# print(day.dtypes)
dataset['Mon'] = (day == 0)*1.0
dataset['Tues'] = (day == 1)*1.0
dataset['Wed'] = (day == 2)*1.0
dataset['Thurs'] = (day == 3)*1.0
dataset['Fri'] = (day == 4)*1.0
dataset['Sat'] = (day == 5)*1.0
dataset['Sun'] = (day == 6)*1.0

dataset['Year'] = pd.DatetimeIndex(dataset['Date']).year
dataset['Month'] = pd.DatetimeIndex(dataset['Date']).month
dataset.head()
df = dataset

In [16]:
dates = df.index
#Split targets and features
X = df.iloc[:,4:14]
Y = df.iloc[:,2]


#Split train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle=True)

#Count features for modelization
X_num_columns= len(X.columns)

#Define model
model = tf.keras.Sequential()

model.add(layers.Dense(300,
                activation='relu',
                input_dim = X_num_columns))

model.add(layers.Dense(90,
                activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(30,
                activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(7,
                activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(1,
                activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')
print("Model Created")

#Fit model to training data
model.fit(X_train, y_train, epochs=5000, batch_size=100)
print("Training completed")

#Save trained model
model.save("Sales_model.h5")
print("Sales_model.h5 saved model to disk in ",os.getcwd())

#Predict known daily sales in order to check results
predictions = model.predict(X)
predictions_list = map(lambda x: x[0], predictions)
predictions_series = pd.Series(predictions_list,index=dates)
dates_series =  pd.Series(dates)

#Import dates to be predicted
df_newDates = pd.read_csv('Upcoming_dates.csv',delimiter=',',index_col='Date')
print("Upcoming dates imported")

#Predict upcoming sales using trained model and imported upcoming dates
Predicted_sales = model.predict(df_newDates)

#Export predicted sales
new_dates_series=pd.Series(df_newDates.index)
new_predictions_list = map(lambda x: x[0], Predicted_sales)
new_predictions_series = pd.Series(new_predictions_list,index=new_dates_series)
new_predictions_series.to_csv("predicted_sales.csv")


W1105 02:18:38.258245 140309402412864 deprecation.py:506] From /home/punishment/Projects/dining_services/env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model Created
Epoch 1/5000


TypeError: float() argument must be a string or a number, not 'Timestamp'